In [3]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import pandas as pd

# 색상 식별

지정된 파일 경로에서 이미지를 읽어와 BGR에서 RGB 형식으로 변환한 후, KMeans 클러스터링을 사용하여 이미지에서 주요한 색상을 식별한다. 

추출할 주요 색상 수를 지정하기 위해 num_colors 매개변수를 조정할 수 있다.

In [4]:
def extract_dominant_colors(image_path, num_colors=35):
    image = cv2.imread(image_path) # OpenCV를 사용하여 이미지를 읽기
    
    # BGR 형식(OpenCV의 기본 형식)에서 RGB 형식으로 변환
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # 이미지를 픽셀 단위로 2D 배열로 변환
    # 각 행은 픽셀을 나타내며 3개의 열(R, G, B)
    pixels = image.reshape(-1, 3)
    
    # KMeans 클러스터링을 적용하여 'num_colors' 개수의 색상 그룹으로 픽셀을 그룹화
    kmeans = KMeans(n_clusters=num_colors)
    kmeans.fit(pixels)
    
    # 클러스터 중심(주요 색상)을 가져와 정수형으로 변환
    dominant_colors = kmeans.cluster_centers_.astype(int)
    
    return dominant_colors

## 데이터프레임 구성

이미지 파일 경로 목록(image_paths)을 받아들여 각 이미지에서 주요 색상을 추출하여 데이터프레임으로 구성한다. num_colors 매개변수를 통해 각 이미지에서 추출할 주요 색상의 수를 지정할 수 있다. 결과로 반환되는 데이터프레임은 각 이미지에 대해 주요 색상 값을 포함하며, 열 이름은 각 주요 색상의 인덱스를 나타낸다.

In [5]:
def create_color_dataset(image_paths, num_colors=35):
    data = {} # 이미지의 주요 색상 데이터를 저장할 딕셔너리
    for path in image_paths:
        # 이미지 경로로부터 이미지 이름 추출
        image_name = path.split('/')[-1].split('.')[0]  
        
        # 주요 색상 추출 함수를 사용하여 각 이미지에서 주요 색상을 추출
        extracted_colors = extract_dominant_colors(path, num_colors)
        
        rgb_arrays = []
        for colors in extracted_colors:
            rgb_arrays.append(colors)
        
        # 이미지 이름을 키로하여 추출된 주요 색상을 데이터 딕셔너리에 저장
        data[image_name] = rgb_arrays
        
    # 데이터 딕셔너리를 Pandas DataFrame으로 변환
    color_df = pd.DataFrame(data).T
    
    # 각 주요 색상에 대한 열 이름을 지정
    color_df.columns = [f'Colors_{i+1}' for i in range(num_colors)]  # Assign column names
    
    return color_df

### 함수 호출

In [10]:
image_paths = ['Color/봄브라이트.png', 'Color/봄웜.png', 'Color/봄라이트.png', 'Color/여름라이트.png', 'Color/여름뮤트.png', 'Color/여름쿨.png', 'Color/가을딥.png', 'Color/가을뮤트.png', 'Color/가을웜.png', 'Color/겨울딥.png', 'Color/겨울브라이트.png', 'Color/겨울쿨.png']  
num_colors_to_extract = 35  # 추출할 주요 색상의 수
color_dataset = create_color_dataset(image_paths, num_colors_to_extract)


transposed_color_dataset = color_dataset.transpose() # 행, 열 변경
transposed_color_dataset.index.name = 'Color' # 인덱스 이름 변경
transposed_color_dataset.columns = [f'{col}_Color' for col in transposed_color_dataset.columns]

/Users/jiyoungjung/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/jiyoungjung/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/jiyoungjung/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/jiyoungjung/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_

In [11]:
transposed_color_dataset

,봄브라이트_Color,봄웜_Color,봄라이트_Color,여름라이트_Color,여름뮤트_Color,여름쿨_Color,가을딥_Color,가을뮤트_Color,가을웜_Color,겨울딥_Color,겨울브라이트_Color,겨울쿨_Color
Color,,,,,,,,,,,,
Colors_1,"[206, 235, 234]","[190, 184, 167]","[253, 253, 253]","[24, 105, 136]","[79, 45, 64]","[119, 130, 184]","[198, 169, 155]","[198, 219, 228]","[184, 175, 170]","[253, 253, 253]","[13, 75, 200]","[193, 202, 210]"
Colors_2,"[197, 42, 95]","[104, 44, 44]","[248, 158, 66]","[213, 192, 155]","[222, 216, 215]","[253, 253, 253]","[60, 18, 36]","[79, 86, 135]","[99, 86, 86]","[65, 33, 49]","[251, 196, 218]","[42, 51, 120]"
Colors_3,"[40, 147, 186]","[242, 239, 235]","[4, 44, 111]","[173, 36, 96]","[197, 141, 167]","[71, 77, 90]","[253, 253, 252]","[128, 129, 110]","[240, 232, 227]","[141, 122, 172]","[132, 70, 82]","[101, 191, 162]"
Colors_4,"[237, 110, 160]","[234, 89, 103]","[152, 168, 176]","[47, 168, 179]","[111, 111, 138]","[190, 71, 112]","[121, 117, 120]","[202, 193, 179]","[190, 156, 128]","[180, 185, 217]","[6, 15, 10]","[252, 253, 252]"
Colors_5,"[60, 125, 62]","[80, 120, 201]","[80, 146, 211]","[239, 232, 232]","[254, 254, 253]","[206, 195, 204]","[188, 110, 78]","[120, 169, 151]","[32, 34, 40]","[93, 95, 106]","[146, 203, 52]","[220, 77, 151]"
Colors_6,"[228, 213, 165]","[63, 54, 60]","[235, 50, 54]","[90, 82, 163]","[152, 153, 174]","[13, 41, 41]","[19, 110, 168]","[214, 115, 135]","[75, 115, 167]","[208, 94, 120]","[60, 200, 244]","[1, 1, 1]"
Colors_7,"[233, 68, 38]","[205, 31, 68]","[246, 176, 169]","[19, 58, 70]","[245, 156, 193]","[155, 144, 146]","[96, 89, 43]","[102, 65, 73]","[211, 82, 66]","[104, 35, 55]","[178, 170, 187]","[147, 131, 195]"
Colors_8,"[253, 210, 93]","[231, 171, 109]","[49, 38, 45]","[183, 118, 143]","[76, 83, 116]","[15, 69, 101]","[190, 37, 68]","[253, 254, 253]","[68, 53, 55]","[206, 203, 212]","[234, 81, 185]","[76, 126, 206]"
Colors_9,"[113, 93, 189]","[127, 188, 102]","[85, 88, 83]","[210, 182, 104]","[136, 108, 113]","[128, 29, 55]","[20, 74, 111]","[46, 86, 100]","[233, 211, 187]","[27, 58, 60]","[20, 126, 53]","[56, 63, 75]"
